# File to parse DrugBanks's drug-drug interactions and severity data #
- 

In [1]:
import pandas as pd
import numpy as np
from toxicity_ranking import *

1. Lowercase for drug names for intersection between DrugComb and DrugBank
2. Get mapping of drug ID, drugbank ID, and drug name, save to file

In [2]:
drugbank_ddi_fp = 'data/DrugBank/u_Brown_DrugBank_CSV/structured_drug_interactions.csv'
drugbank_ddi_descriptions_fp = 'data/DrugBank/u_Brown_DrugBank_CSV/drug_interactions.csv'
drugbank_ddi_df = pd.read_csv(drugbank_ddi_fp)

# Answer a few questions about the dataset to begin with:
print('How many levels of severity are present in the drugbank database?')
print(len(drugbank_ddi_df['severity'].unique()))

print('Are there any NA values in the severity column?')
print(drugbank_ddi_df['severity'].isna().sum())

print('How many interactions are in each severity category in the drugbank database?')
print(drugbank_ddi_df['severity'].value_counts())

print("What are the types of actions resulting from the DDI?")
print(drugbank_ddi_df['action'].unique())

# make all drug names lowercase
drugbank_ddi_df['subject_drug_name'] = drugbank_ddi_df['subject_drug_name'].str.lower()
drugbank_ddi_df['affected_drug_name'] = drugbank_ddi_df['affected_drug_name'].str.lower()
print(drugbank_ddi_df.columns)

# create a dataframe mapping 
subject_drug_df = drugbank_ddi_df[['subject_drug_id', 'subject_drug_drugbank_id', 'subject_drug_name']]
subject_drug_df = subject_drug_df.drop_duplicates()
subject_drug_df = subject_drug_df.reset_index(drop=True)
subject_drug_df.columns = ['drug_id', 'drugbank_id', 'drug_name']
affected_drug_df = drugbank_ddi_df[['affected_drug_id', 'affected_drug_drugbank_id', 'affected_drug_name']]
affected_drug_df = affected_drug_df.drop_duplicates()
affected_drug_df = affected_drug_df.reset_index(drop=True)
affected_drug_df.columns = ['drug_id', 'drugbank_id', 'drug_name']
drugbank_drug_ids_to_names = pd.concat([subject_drug_df, affected_drug_df], axis=0)
drugbank_drug_ids_to_names = drugbank_drug_ids_to_names.drop_duplicates()
drugbank_drug_ids_to_names = drugbank_drug_ids_to_names.reset_index(drop=True)
drugbank_drug_ids_to_names.to_csv('data_processed/drugbank_drug_ids_to_names.csv', index=False)

/var/folders/6g/w3wk1pc96f70nng8j2j_0c440000gn/T/ipykernel_274/1472229666.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  drugbank_ddi_df = pd.read_csv(drugbank_ddi_fp)


How many levels of severity are present in the drugbank database?
3
Are there any NA values in the severity column?
0
How many interactions are in each severity category in the drugbank database?
severity
1    742987
0    535009
2    152458
Name: count, dtype: int64
What are the types of actions resulting from the DDI?
['increase_specific_adverse_effects' 'increase_adverse_effects'
 'increase_specific_effects' 'decrease_therapeutic_efficacy'
 'increase_therapeutic_efficacy' 'increase_serum_concentration'
 'decrease_serum_concentration' 'increase_dynamics' 'decrease_dynamics'
 'decrease_excretion' 'increase_absorption' 'decrease_absorption'
 'decrease_diagnostic_effectiveness' 'decrease_specific_effects'
 'increase_risk_of_hypersensitivity' 'decrease_active_metabolites'
 'decrease_specific_active_metabolites' 'increase_active_metabolites'
 'decrease_adverse_effects' 'increase_excretion'
 'increase_specific_active_metabolites'
 'decrease_specific_adverse_effects' 'increase_diagnostic_eff

1. Get the intersection between the drugbank_ddi_df and DrugComb dataset
    - Based on measurements of severity and the descriptions, determined that 0 is lowest severity and 2 is high severity

In [3]:
# Get the intersection of the drugcomb data and the drugbank data
drugcomb_df = get_drug_comb_data(bliss=True, loewe=True, hsa=True, zip=True)
drugbank_syntox_df, intersection_major_pairs, intersection_moderate_pairs, intersection_minor_pairs, intersection_unknown_pairs = find_drugcomb_drugbankddi_intersect(drugcomb_df, drugbank_ddi_df)

/Users/alexwong/Research/toxicity-cancer-drug-combination/toxicity_ranking.py:16: DtypeWarning: Columns (2,7,19,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  drugcomb_df = pd.read_csv('data/DrugComb/drugcomb_summary_v_1_5.csv', sep=',', index_col=False)


Original shape of drugcomb data:  (1432351, 26)
Final shape of filtered drugcomb data:  (722032, 26)
Number of drugs in common between drugcomb and drugbank [lowercase enforced]:  1079
Major pairs in both DrugComb and in DrugBank:  3211
Moderate pairs in both DrugComb and in DrugBank:  5534
Minor pairs in both DrugComb and in DrugBank:  4799
Unknown toxicity pairs in both DrugComb and in DrugBank:  56177
Total common pairs:  69721
Total known pairs:  13544


In [4]:
drugbank_syntox_df = drugbank_syntox_df[drugbank_syntox_df['toxicity_category'] != 'Unknown']
drugbank_syntox_df.to_csv('data_processed/drugbank_syntox_known.csv', index=False)

In [31]:
# How many of the drugcomb-ddinter intersection pairs overlap with the drugcomb-ddinter intersection pairs?
ddinter_syntox_df = pd.read_csv('data_processed/syntox_known.csv')
num_agreeing_pairs = 0
ddinter_pairs_not_drugbank = set()
drugbank_pairs_not_ddinter = set()
disagreeing_pairs = set() # drug 1, drug 2, drugbank_toxicity, ddinter_toxicity

# create a set of all the drug pairs in the drugbank data (flipped and unflipped)
drugbank_pairs = {}
for index, row in drugbank_syntox_df.iterrows():
    drug1 = row['drug_row']
    drug2 = row['drug_col']
    drugbank_tox = row['toxicity_category']
    if (drug1, drug2) not in drugbank_pairs and (drug2, drug1) not in drugbank_pairs:
        drugbank_pairs[(drug1, drug2)] = drugbank_tox
        drugbank_pairs[(drug2, drug1)] = drugbank_tox

ddinter_pairs = {}
for index, row in ddinter_syntox_df.iterrows():
    drug1 = row['drug_row']
    drug2 = row['drug_col']
    ddinter_tox = row['toxicity_category']
    if (drug1, drug2) in drugbank_pairs or (drug2, drug1) in drugbank_pairs:
        if drugbank_pairs[(drug1, drug2)] != ddinter_tox:
            disagreeing_pairs.add((drug1, drug2, drugbank_pairs[(drug1, drug2)], ddinter_tox))
        else:
            num_agreeing_pairs += 1
    else:
        ddinter_pairs_not_drugbank.add((drug1, drug2))
    
    if (drug1, drug2) not in ddinter_pairs and (drug2, drug1) not in ddinter_pairs:
        ddinter_pairs[(drug1, drug2)] = ddinter_tox
        ddinter_pairs[(drug2, drug1)] = ddinter_tox

for index, row in drugbank_syntox_df.iterrows():
    drug1 = row['drug_row']
    drug2 = row['drug_col']
    if (drug1, drug2) not in ddinter_pairs and (drug2, drug1) not in ddinter_pairs:
        drugbank_pairs_not_ddinter.add((drug1, drug2))

print('Number of drug pairs in the intersection of drugcomb and drugbank that agree on toxicity category:', num_agreeing_pairs)
print('Number of drug pairs in the intersection of drugcomb and drugbank that disagree on toxicity category:', len(disagreeing_pairs))
print('Number of drug pairs in the intersection of drugcomb and drugbank that are not in the ddinter data:', len(drugbank_pairs_not_ddinter))
print('Number of drug pairs in the intersection of drugcomb and ddinter that are not in the drugbank data:', len(ddinter_pairs_not_drugbank))
print(disagreeing_pairs) # There are pairs that disagree on toxicity category between DrugBank and DDInter

Number of drug pairs in the intersection of drugcomb and drugbank that agree on toxicity category: 9466
Number of drug pairs in the intersection of drugcomb and drugbank that disagree on toxicity category: 1546
Number of drug pairs in the intersection of drugcomb and drugbank that are not in the ddinter data: 10835
Number of drug pairs in the intersection of drugcomb and ddinter that are not in the drugbank data: 595
{('temozolomide', 'docetaxel', 'Major', 'Moderate'), ('dexamethasone', 'sorafenib', 'Major', 'Moderate'), ('ruxolitinib', 'nilotinib', 'Moderate', 'Minor'), ('flutamide', 'halofantrine', 'Minor', 'Major'), ('risperidone', 'sunitinib', 'Major', 'Moderate'), ('tamoxifen', 'sunitinib', 'Major', 'Moderate'), ('nabilone', 'spironolactone', 'Minor', 'Moderate'), ('bortezomib', 'mephenytoin', 'Moderate', 'Minor'), ('lapatinib', 'erlotinib', 'Major', 'Moderate'), ('imatinib', 'ibrutinib', 'Moderate', 'Major'), ('lumefantrine', 'ranolazine', 'Moderate', 'Major'), ('rufinamide', 'bu